<a href="https://colab.research.google.com/github/anaustinbeing/neural-networks/blob/main/keras_sequential_finetuning_(diabetes).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing a multi-layer ANN on diabetes dataset using Keras Sequential and finetuning hyperparameters.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
np.random.seed(42)

In [ ]:
df = pd.read_csv('diabetes_noheaders.csv', header=None)

In [ ]:
X, y = df.values[:, :-1], df.values[:, -1]
X = X.astype('float32')

Splitting data into training and testing:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(514, 8) (254, 8) (514,) (254,)


In [ ]:
n_features = X_train.shape[1]
n_features

8

In [ ]:
model = keras.models.Sequential()
# input layer with 8 neurons
model.add(keras.layers.Dense(n_features))
# add the first hidden layer with 32 neurons, relu activation function
model.add(keras.layers.Dense(32, activation="relu", kernel_initializer='he_normal'))
# add the second hidden layer with 16 neurons, relu activation function
model.add(keras.layers.Dense(16, activation="relu", kernel_initializer='he_normal'))
# add the output layer with 1 neurons, sigmoid activation function
model.add(keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=150, batch_size=32, verbose=0)

In [ ]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: %.3f' % acc)

Test Accuracy: 0.681


In [ ]:
predictions = model.predict(X_test)
predictions = np.array([np.round(x) for x in predictions])
print('The original test target value is: ', y_test[0])
print('The predicted test value is: ', predictions[0])

print('\nTotal predictions: ')
print(len(predictions))

print('Count of predictions that are correct: ')
print(sum(predictions[i] == y_test[i] for i in range(len(predictions)))[0])

print('Count of predictions that are not correct: ')
print(sum(predictions[i] != y_test[i] for i in range(len(predictions)))[0])

8/8 [==============================] - 0s 2ms/step
The original test target value is:  0.0
The predicted test value is:  [1.]

Total predictions: 
254
Count of predictions that are correct: 
173
Count of predictions that are not correct: 
81


In [ ]:
for i in range(5):
  print('For input: ', X_test[i], ', the actual output is ', y_test[i])
  print('Predicted output is ', predictions[i][0])
  if predictions[i][0] == 1.0:
    print("Patient has developed diabetes within five years of the initial measurement.")
  else:
    print("Patient has not developed diabetes within five years of the initial measurement.")
  print()

For input:  [  6.    98.    58.    33.   190.    34.     0.43  43.  ] , the actual output is  0.0
Predicted output is  1.0
Patient has developed diabetes within five years of the initial measurement.

For input:  [  2.    112.     75.     32.      0.     35.7     0.148  21.   ] , the actual output is  0.0
Predicted output is  0.0
Patient has not developed diabetes within five years of the initial measurement.

For input:  [  2.    108.     64.      0.      0.     30.8     0.158  21.   ] , the actual output is  0.0
Predicted output is  0.0
Patient has not developed diabetes within five years of the initial measurement.

For input:  [  8.    107.     80.      0.      0.     24.6     0.856  34.   ] , the actual output is  0.0
Predicted output is  0.0
Patient has not developed diabetes within five years of the initial measurement.

For input:  [  7.   136.    90.     0.     0.    29.9    0.21  50.  ] , the actual output is  0.0
Predicted output is  0.0
Patient has not developed diabetes wi

We see that most of the predictions are correct.

### FineTuning:

We are finetuning the number of hidden layers and the number of neurons in the hidden layers.

In [ ]:
def build_model(n_hidden=2, n_neurons=32):
  model = keras.models.Sequential()
  # input layer
  model.add(keras.layers.Dense(n_features))
  for hidden_layer in range(n_hidden):
    # add the first hidden layer with 10 neurons, relu activation function
    model.add(keras.layers.Dense(n_neurons, activation="relu", kernel_initializer='he_normal'))
  # add the output layer with 10 neurons, erlu activation function
  model.add(keras.layers.Dense(1, activation="sigmoid"))
  model.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])
  return model

In [ ]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV

param_distribs = {
    "n_hidden": list(range(30)),
    "n_neurons": np.arange(1, 200),
}

keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=1, cv=2, verbose=2)
rnd_search_cv.fit(X_train, y_train)
rnd_search_cv.best_params_

Fitting 2 folds for each of 1 candidates, totalling 2 fits


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  if sys.path[0] == '':


9/9 [==============================] - 0s 2ms/step - loss: 6.9329 - accuracy: 0.5914
[CV] END .........................n_hidden=10, n_neurons=139; total time=   1.3s
9/9 [==============================] - 0s 3ms/step - loss: 3.6098 - accuracy: 0.6809
[CV] END .........................n_hidden=10, n_neurons=139; total time=   1.2s
17/17 [==============================] - 1s 6ms/step - loss: 8.7361 - accuracy: 0.5759


{'n_neurons': 139, 'n_hidden': 10}

After the `RandomizedSearchCV` fit, we get the best parameters from the range we have provided.

Printing the best parameters below:

In [ ]:
best_params = rnd_search_cv.best_params_
best_params

{'n_neurons': 139, 'n_hidden': 10}

Now we will build the model again with the best parameters we found out in the previous step.

In [ ]:
def build_model(n_hidden=2, n_neurons=32):
  model = keras.models.Sequential()
  # input layer
  model.add(keras.layers.Dense(n_features))
  for hidden_layer in range(n_hidden):
    # add the first hidden layer with 10 neurons, relu activation function
    model.add(keras.layers.Dense(n_neurons, activation="relu", kernel_initializer='he_normal'))
  # add the output layer with 10 neurons, erlu activation function
  model.add(keras.layers.Dense(1, activation="sigmoid"))
  model.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])
  return model

best_model = build_model(best_params['n_hidden'], best_params['n_neurons'])
best_model.fit(X_train, y_train, epochs=150, batch_size=32, verbose=0)

In [ ]:
loss, acc = best_model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: %.3f' % acc)

Test Accuracy: 0.752


We see that the accuracy is greater after the finetuning.

In [ ]:
best_predictions = best_model.predict(X_test)
best_predictions = np.array([np.round(x) for x in best_predictions])
print('The original test target value is: ', y_test[0])
print('The predicted test value is: ', best_predictions[0])

print('\nTotal predictions: ')
print(len(best_predictions))

print('Count of predictions that are correct: ')
print(sum(best_predictions[i] == y_test[i] for i in range(len(best_predictions)))[0])

print('Count of predictions that are not correct: ')
print(sum(best_predictions[i] != y_test[i] for i in range(len(best_predictions)))[0])

8/8 [==============================] - 0s 3ms/step
The original test target value is:  0.0
The predicted test value is:  [1.]

Total predictions: 
254
Count of predictions that are correct: 
191
Count of predictions that are not correct: 
63


In [ ]:
for i in range(5):
  print('For input: ', X_test[i], ', the actual output is ', y_test[i])
  print('Predicted output is ', best_predictions[i][0])
  if best_predictions[i][0] == 1.0:
    print("Patient has developed diabetes within five years of the initial measurement.")
  else:
    print("Patient has not developed diabetes within five years of the initial measurement.")
  print()

For input:  [  6.    98.    58.    33.   190.    34.     0.43  43.  ] , the actual output is  0.0
Predicted output is  1.0
Patient has developed diabetes within five years of the initial measurement.

For input:  [  2.    112.     75.     32.      0.     35.7     0.148  21.   ] , the actual output is  0.0
Predicted output is  0.0
Patient has not developed diabetes within five years of the initial measurement.

For input:  [  2.    108.     64.      0.      0.     30.8     0.158  21.   ] , the actual output is  0.0
Predicted output is  0.0
Patient has not developed diabetes within five years of the initial measurement.

For input:  [  8.    107.     80.      0.      0.     24.6     0.856  34.   ] , the actual output is  0.0
Predicted output is  0.0
Patient has not developed diabetes within five years of the initial measurement.

For input:  [  7.   136.    90.     0.     0.    29.9    0.21  50.  ] , the actual output is  0.0
Predicted output is  1.0
Patient has developed diabetes within

The above is a friendly format of the input, the actual output and the predicted output.